In [14]:
def deco(f):
    pass

In [15]:
@deco
def test():
    pass

In [16]:
test()

TypeError: 'NoneType' object is not callable

In [4]:
def test2():
    pass

In [5]:
test2

<function __main__.test2>

In [6]:
test2 = deco(test2)

In [7]:
test2

In [8]:
test2 is test

True

In [9]:
def invalid_deco(): # 引数を受け取らないデコレータは無効
    pass

In [10]:
@invalid_deco
def test():
    pass

TypeError: invalid_deco() takes 0 positional arguments but 1 was given

In [11]:
invalid_deco2 = 2

In [12]:
@invalid_deco2
def test():
    pass

TypeError: 'int' object is not callable

In [18]:
def deco2(f):
    def _wrap(*args, **kwargs):
        print('前処理')
        f(*args, **kwargs)
        print('後処理')
    return _wrap

In [19]:
@deco2
def test2():
    """大事な処理"""
    print('メイン処理')

In [20]:
test2()

前処理
メイン処理
後処理


In [28]:
def test3():
    """大事な処理"""
    pass

In [32]:
print(test3, test3.__name__, test3.__doc__)

<function test3 at 0x7f44067a01e0> test3 大事な処理


In [33]:
print(test2, test2.__name__, test2.__doc__)

<function deco2.<locals>._wrap at 0x7f44067860d0> _wrap None


In [34]:
import functools

In [35]:
def deco3(f):
    @functools.wraps(f)
    def _wrap(*args, **kwargs):
        print('前処理')
        f(*args, **kwargs)
        print('後処理')
    return _wrap

In [36]:
@deco3
def test4():
    """大事な処理"""
    pass

In [38]:
print(test4, test4.__name__, test4.__doc__)

<function test4 at 0x7f44067b4048> test4 大事な処理


In [40]:
_deco = functools.wraps(test3)

In [44]:
test4 is  _deco(test4) # functools.wrapsで作られたデコレータは同じオブジェクトを返す

True

In [46]:
# enclosing function's scopeの変数を使うデコレータ
def deco4(f):
    val1 = 'enclosing1'
    def _wrap(*args, **kwargs):
        print(val1)
        f(*args, **kwargs)
        print(val2)
    val2 = 'enclosing2'
    return _wrap

In [47]:
@deco4
def test5():
    print(1)

In [48]:
test5()

enclosing1
1
enclosing2


In [49]:
def deco5(f):
    i = 0
    def _wrap(*args, **kwargs):
        i += 1
        # i = i + 1 # と同じ。変数の上書きがあるとPythonはローカル変数と判断するのでエラーになる
        print('{0}回目の実行'.format(i))
        f(*args, **kwargs)
    return _wrap

In [50]:
@deco5
def test6():
    print('a')

In [51]:
test6()

UnboundLocalError: local variable 'i' referenced before assignment

In [52]:
def deco6(f):
    i = 0
    def _wrap(*args, **kwargs):
        nonlocal i
        i += 1
        # i = i + 1 # と同じ。変数の上書きがあるとPythonはローカル変数と判断するのでエラーになる
        print('{0}回目の実行'.format(i))
        f(*args, **kwargs)
    return _wrap

In [53]:
@deco6
def test7():
    print('a')

In [56]:
test7()

2回目の実行
a


In [57]:
@deco6
def test8():
    print('b')

In [58]:
test8()

1回目の実行
b


In [59]:
def deco7(f):
    i = [0]
    # nonlocalを使わない場合、ミュータブルなオブジェクト書き換えでも対応できる
    # でもふつうはクラスを使いましょう
    def _wrap(*args, **kwargs):
        i[0] += 1
        print('{0}回目の実行'.format(i[0]))
        f(*args, **kwargs)
    return _wrap

In [60]:
@deco7
def test9():
    print('c')

In [61]:
test9()

1回目の実行
c


In [78]:
def deco8(count):
    print('{0}回繰り返し実行します'.format(count))
    def _deco(f):
        def _wrap(*args, **kwargs):
            # 最外で受け取った引数をここでも参照できる
            for i in range(count):
                print([i], end='')
                f(*args, **kwargs)
        return _wrap
    return _deco

In [81]:
@deco8(1)
def test10():
    print(10)

test10()

1回繰り返し実行します
[0]10


In [82]:
@deco8(2)
def test11():
    print(11)

test11()

2回繰り返し実行します
[0]11
[1]11


In [83]:
# 一つ目のかっこの後ろは何も書けない(その１)
@deco8(2)()
def test():
    pass

SyntaxError: invalid syntax (<ipython-input-83-5db665cdd70e>, line 2)

In [84]:
# 一つ目のかっこの後ろは何も書けない(その２)
@deco8(2).a
def test():
    pass

SyntaxError: invalid syntax (<ipython-input-84-8ad64d3c6076>, line 2)

In [86]:
def cls_deco(cls):
    cls.value = 100
    return cls
 
@cls_deco
class Test(object):
    pass

In [87]:
test = Test()
test.value

100

In [89]:
# 課題: 指定した日時以降に適用した関数が実行できなくなるデコレータを作ってください

In [96]:
# 解答例
from datetime import datetime
from functools import wraps


class TimeupError(BaseException):
    pass


def timelimit(limit):
    def deco(f):
        @wraps(f)
        def _wrap(*args, **kwargs):
            if datetime.now() >= limit:
                raise TimeupError('timeup!!')
            return f(*args, **kwargs)
        return _wrap
    return deco

In [97]:
@timelimit(limit=datetime.now())
def test_ng():
    print('ok')

In [98]:
@timelimit(limit=datetime(9999, 12, 31))
def test_ok():
    print('ok')

In [100]:
test_ok()

ok


In [101]:
test_ng()

TimeupError: timeup!!